In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load test dataset
df = pd.read_csv("Small_Retail_Sales_Test_Dataset.csv")

# Preprocessing
df["DATE"] = pd.to_datetime(df["DATE"])
df["DAY"] = df["DATE"].dt.day
df["DAY_OF_WEEK"] = df["DATE"].dt.dayofweek
df["IS_WEEKEND"] = df["DAY_OF_WEEK"].apply(lambda x: 1 if x >= 5 else 0)
df.drop(columns=["ITEM CODE", "ITEM DESCRIPTION"], inplace=True)

# Handle categorical and boolean
categorical_cols = ["SUPPLIER", "weather_condition"]
df[categorical_cols] = df[categorical_cols].astype(str)
bool_cols = df.select_dtypes(include='bool').columns
df[bool_cols] = df[bool_cols].astype(int)

# Target and features
X = df.drop(columns=["RETAIL SALES"])
y = df["RETAIL SALES"]

# One-hot encode
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Save DATE if still there
if 'DATE' in X.columns:
    X.drop(columns=['DATE'], inplace=True)

# Split and scale (just for demo; in practice you'd use real train data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train XGBoost with your tuned params
xgb_model = xgb.XGBRegressor(
    learning_rate=0.1,
    max_depth=6,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

xgb_model.fit(X_train_scaled, y_train)

# Predict and evaluate
y_pred = xgb_model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"\n🚀 XGBoost Evaluation:")
print(f"MAE: {mae:.6f}")
print(f"MSE: {mse:.6f}")
print(f"RMSE: {rmse:.6f}")
print(f"R² Score: {r2:.6f}")


🚀 XGBoost Evaluation:
MAE: 0.019265
MSE: 0.002498
RMSE: 0.049975
R² Score: 0.813152
